# Edison platform client usage example

In [ ]:
import os

from edison_client import (
    EdisonClient,
    JobNames,
    TaskRequest,
)
from edison_client.models import RuntimeConfig
from ldp.agent import AgentConfig

## Client instantiation

Here we instantiate an Edison client and authenticate our access to the platform.
By default, the client will use the `EDISON_PLATFORM_API_KEY` environment variable to authenticate.
The option `api_key` can be used to pass your API key.

Please log in to the platform and go to your user settings to get your API key.

In [ ]:
client = EdisonClient(
    api_key=os.getenv("EDISON_PLATFORM_API_KEY", "your-api-key"),
)

## Submit a task to an available Edison job

In the edison platform, we refer to the deployed combination of agent and environment as a `job`.
Submitting a task to an edison job is done by calling the `create_task` method, which receives a `TaskRequest` object.

For convenience, one can use the `run_tasks_until_done` method, which submits the task, waits for the task to be completed, and returns a list of `TaskResponse` objects.

In [ ]:
task_data = TaskRequest(
    name=JobNames.from_string("literature"),
    query="What is the molecule known to have the greatest solubility in water?",
)
responses = client.run_tasks_until_done(task_data)
task_response = responses[0]

print(f"Job status: {task_response.status}")
print(f"Job answer: \n{task_response.formatted_answer}")

You can also pass a `runtime_config` to the `run_tasks_until_done` method, which will be used to configure the agent on runtime.
Here, we will define a agent configuration and include it in the `TaskRequest`. This agent is used to decide the next action to take.
We will also use the `max_steps` parameter to limit the number of steps the agent will take.

In [ ]:
agent = AgentConfig(
    agent_type="SimpleAgent",
    agent_kwargs={
        "model": "gpt-4o",
        "temperature": 0.0,
    },
)
task_data = TaskRequest(
    name=JobNames.LITERATURE,
    query="How many moons does earth have?",
    runtime_config=RuntimeConfig(agent=agent, max_steps=10),
)
responses = client.run_tasks_until_done(task_data)
task_response = responses[0]

print(f"Job status: {task_response.status}")
print(f"Job answer: \n{task_response.formatted_answer}")

# Continue a job

The platform allows to ask follow-up questions to the previous job.
To accomplish that, we can use the `runtime_config` to pass the `task_id` of the previous task.

Notice that `run_tasks_until_done` accepts both a `TaskRequest` object and a dictionary with keywords arguments.

In [ ]:
task_data = TaskRequest(
    name=JobNames.LITERATURE, query="How many species of birds are there?"
)

responses = client.run_tasks_until_done(task_data)
task_response = responses[0]

print(f"First job status: {task_response.status}")
print(f"First job answer: \n{task_response.formatted_answer}")

In [ ]:
continued_job_data = {
    "name": JobNames.LITERATURE,
    "query": "From the previous answer, specifically, how many species of crows are there?",
    "runtime_config": {"continued_job_id": task_response.task_id},
}

responses = client.run_tasks_until_done(continued_job_data)
continued_task_response = responses[0]


print(f"Continued job status: {continued_task_response.status}")
print(f"Continued job answer: \n{continued_task_response.formatted_answer}")